# HyDesign Sizing Example

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DTUWindEnergy/HyDesignExamples/blob/main/notebooks/Simple_Sizing_Example.ipynb)[![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/DTUWindEnergy/HyDesignExamples/blob/main/notebooks/Simple_Sizing_Example.ipynb)

In [ ]:
# Detect if running in Kaggle
import os 
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE'):
    mypaths = !python -c "import sysconfig; print(sysconfig.get_paths()['purelib'])"
    mypath = mypaths[0]
    !pip install trash-cli
    !trash $mypath/numpy*
    !pip install --upgrade numpy
    !pip install finitediff
    import os
    os.kill(os.getpid(), 9)

In [ ]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git   

In [1]:
from hydesign.hpp_assembly import hpp_model
from hydesign.Parallel_EGO import get_kwargs, EfficientGlobalOptimizationDriver

In [2]:
# Simple example to size wind only with a single core to run test machines and colab

inputs = {
    'example': 4,
    'name': None,
    'longitude': None,
    'latitude': None,
    'altitude': None,
    'input_ts_fn': None,
    'sim_pars_fn': None,

    'opt_var': "NPV_over_CAPEX",
    'num_batteries': 1,
    'n_procs': 1,
    'n_doe': 8,
    'n_clusters': 1,
    'n_seed': 0,
    'max_iter': 2,
    'final_design_fn': 'hydesign_design_0.csv',
    'npred': 3e4,
    'tol': 1e-6,
    'min_conv_iter': 2,
    'work_dir': './',
    'hpp_model': hpp_model,
    }

kwargs = get_kwargs(inputs)
kwargs['variables'] = {
    'clearance [m]':
        {'var_type':'design',
          'limits':[10, 60],
          'types':'int'
          },
        # {'var_type':'fixed',
        #   'value': 35
        #   },
     'sp [W/m2]':
        {'var_type':'design',
         'limits':[200, 360],
         'types':'int'
         },
    'p_rated [MW]':
        {'var_type':'design',
          'limits':[1, 10],
          'types':'int'
          },
        # {'var_type':'fixed',
        #  'value': 6
         # },
    'Nwt':
        {'var_type':'design',
          'limits':[0, 400],
          'types':'int'
          },
        # {'var_type':'fixed',
        #   'value': 200
        #   },
    'wind_MW_per_km2 [MW/km2]':
        {'var_type':'design',
          'limits':[5, 9],
          'types':'float'
          },
        # {'var_type':'fixed',
        #   'value': 7
        #   },
    'solar_MW [MW]':
        # {'var_type':'design',
        #   'limits':[0, 400],
        #   'types':'int'
        #   },
        {'var_type':'fixed',
          'value': 200
          },
    'surface_tilt [deg]':
        # {'var_type':'design',
        #   'limits':[0, 50],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value': 25
          },
    'surface_azimuth [deg]':
        # {'var_type':'design',
        #   'limits':[150, 210],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value': 180
          },
    'DC_AC_ratio':
        # {'var_type':'design',
        #   'limits':[1, 2.0],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value':1.0,
          },
    'b_P [MW]':
        # {'var_type':'design',
        #   'limits':[0, 100],
        #   'types':'int'
        #   },
        {'var_type':'fixed',
          'value': 50
          },
    'b_E_h [h]':
        # {'var_type':'design',
        #   'limits':[1, 10],
        #   'types':'int'
        #   },
        {'var_type':'fixed',
          'value': 6
          },
    'cost_of_battery_P_fluct_in_peak_price_ratio':
        # {'var_type':'design',
        #   'limits':[0, 20],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value': 10},
    }
EGOD = EfficientGlobalOptimizationDriver(**kwargs)

Selected example site:
---------------------------------------------------
case                                         Europe
name                               France_good_wind
longitude                                 -0.864258
latitude                                  48.744116
altitude                                      302.0
input_ts_fn    Europe/input_ts_France_good_wind.csv
sim_pars_fn                     Europe/hpp_pars.yml
price_fn                   Europe/2030-EL_PRICE.csv
price_col                                      FR_R
Name: 4, dtype: object


In [3]:
EGOD.run()





Sizing a HPP plant at France_good_wind:


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0



Initial 8 simulations took 1.77 minutes
  Current solution -NPV_over_CAPEX = 1.407E-01

Update sm and extract candidate points took 0.07 minutes
Check-optimal candidates: new 11 simulations took 2.34 minutes
  Current solution -NPV_over_CAPEX = 2.524E-02
  rel_yopt_change = 4.57E+00
Iteration 1 took 2.42 minutes

Update sm and extract candidate points took 0.08 minutes
Check-optimal candidates: new 11 simulations took 2.58 minutes
  Current solution -NPV_over_CAPEX = -1.887E-02
  rel_yopt_change = -2.34E+00
Iteration 2 took 2.66 minutes


Design:
---------------
clearance [m]: 60.000
sp [W/m2]: 250.000
p_rated [MW]: 10.000
Nwt: 25.000
wind_MW_per_km2 [MW/km2]: 5.750
solar_MW [MW]: 200.000
surface_tilt [deg]: 25.000
surface_azimuth [deg]: 180.000
DC_AC_ratio: 1.000
b_P [MW]: 50.000
b_E_h [h]: 6.000
cost_of_battery_P_flu

In [5]:
EGOD.result

,clearance [m],sp [W/m2],p_rated [MW],Nwt,wind_MW_per_km2 [MW/km2],solar_MW [MW],surface_tilt [deg],surface_azimuth [deg],DC_AC_ratio,b_P [MW],...,Battery Power [MW],Total curtailment [GWh],Awpp [km2],Apvp [km2],Rotor diam [m],Hub height [m],Number of batteries used in lifetime,design obj,opt time [min],n_model_evals
France_good_wind,60.0,250.0,10.0,25.0,5.75,200.0,25.0,180.0,1.0,50.0,...,50.0,112.757833,43.478261,2.452,225.675833,172.837917,1.0,NPV_over_CAPEX,7.07,28
